# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [32]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_csv_path = "../WeatherPy/Output/WeatherPy_data.csv"
weather_data_df = pd.read_csv(city_csv_path)

weather_data = weather_data_df.drop(["Unnamed: 0"], axis = 1)
weather_data


City      Lat       Lng  Max_Temp  Humidity  Cloudiness  \
0            seoul  37.5683  126.9778     81.84        57          29   
1    nyazepetrovsk  56.0531   59.6028     66.74        65          99   
2           tsogni  -2.7942   10.0988     76.77        64          24   
3       esmeraldas -19.7625  -44.3139     68.86        41           0   
4        taoudenni  22.6783   -3.9836    113.14         6           7   
..             ...      ...       ...       ...       ...         ...   
533    bella union -30.2752  -57.6007     70.52        58          69   
534          bumba   2.1833   22.4667     86.00        49          98   
535       necochea -38.5473  -58.7368     51.04        57          98   
536       uinskoye  56.8817   56.5813     62.92        82          79   
537       chapleau  47.8405  -83.4034     68.09        94          90   

     Wind_Speed Country        Date  
0          1.45      KR  1627135960  
1          2.30      RU  1627136044  
2         10.42      GA  1627136045  
3          8.46      BR  1627136045  
4         10.29      ML  1627136045  
..          ...     ...         ...  
533       11.74      UY  1627136198  
534        3.78      CD  1627136166  
535       15.10      AR  1627136199  
536        5.12      RU  1627136199  
537       10.36      CA  1627136199  

[538 rows x 9 columns]

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [7]:
city_loc = weather_data[["Lat", "Lng"]]
humid = weather_data["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(city_loc, weights=humid, 
                                 dissipating=False, max_intensity=np.max(humid),
                                 point_radius=3)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
perfect_weather = weather_data.loc[weather_data["Max_Temp"] < 80, :]
perfect_weather = perfect_weather.loc[perfect_weather["Max_Temp"] > 70, :]
perfect_weather = perfect_weather.loc[perfect_weather["Wind_Speed"] < 10, :]
perfect_weather = perfect_weather.loc[perfect_weather["Cloudiness"] == 0, :]

perfect_weather = perfect_weather.dropna()

perfect_weather

City      Lat       Lng  Max_Temp  Humidity  Cloudiness  \
21           albany  42.6001  -73.9662     75.15        73           0   
91          labuhan  -6.8844  112.2051     78.42        73           0   
282     santa maria -29.6842  -53.8069     76.30        44           0   
335          namibe -15.1961   12.1522     74.71        54           0   
375   bambanglipuro  -7.9500  110.2833     76.30        79           0   
420  preobrazheniye  42.9019  133.9064     74.07        91           0   
458          hasaki  35.7333  140.8333     77.40        82           0   

     Wind_Speed Country        Date  
21         1.01      US  1627136050  
91         5.70      ID  1627136071  
282        2.30      BR  1627136130  
335        7.65      AO  1627136143  
375        8.46      ID  1627136153  
420        7.02      RU  1627136167  
458        5.64      JP  1627136178

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [37]:
hotel_df = perfect_weather
hotel_df["Hotel Name"] = ""
hotel_df


target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [41]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    params["location"] = f"{lat}, {lng}"
    
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except:
        print("Zero Result")
        hotel_df.loc[index, "Hotel Name"] = "No Result"
        
hotel_df

Zero Result
Zero Result
Zero Result


City      Lat       Lng  Max_Temp  Humidity  Cloudiness  \
21           albany  42.6001  -73.9662     75.15        73           0   
91          labuhan  -6.8844  112.2051     78.42        73           0   
282     santa maria -29.6842  -53.8069     76.30        44           0   
335          namibe -15.1961   12.1522     74.71        54           0   
375   bambanglipuro  -7.9500  110.2833     76.30        79           0   
420  preobrazheniye  42.9019  133.9064     74.07        91           0   
458          hasaki  35.7333  140.8333     77.40        82           0   

     Wind_Speed Country        Date              Hotel Name  
21         1.01      US  1627136050               No Result  
91         5.70      ID  1627136071               No Result  
282        2.30      BR  1627136130             Hotel Appel  
335        7.65      AO  1627136143  Hotel Chik Chik Namibe  
375        8.46      ID  1627136153               No Result  
420        7.02      RU  1627136167          Campsite Orlan  
458        5.64      JP  1627136178       Villa Umi To Mori

In [39]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [40]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))